# Part 2.3

## Libraries to import

In [ ]:
import spacy
from spacy.language import Language

In [ ]:
import stanza
from stanza.server import CoreNLPClient
stanza.download('en')
stanza_nlp = stanza.Pipeline(lang='en', processors='tokenize,ner')

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: en (English) ...


INFO:stanza:Downloaded file to /root/stanza_resources/en/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: en (English):
| Processor | Package                   |
-----------------------------------------
| tokenize  | combined                  |
| mwt       | combined                  |
| ner       | ontonotes-ww-multi_charlm |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


In [ ]:
from spacy.pipeline import EntityRecognizer
import pandas as pd
nlp = spacy.load("en_core_web_sm")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import json

In [ ]:
import re

## Verification against KG

In [ ]:
#extract the text from json
with open("./preproc_texts.json", "r") as f:
  data = json.load(f)

In [ ]:
#separate the two keys
Female_mathematicians = data["Female_mathematicians"]
Contemporary_artists = data["Contemporary_artists"]

In [ ]:
print(Female_mathematicians) #check
print(Contemporary_artists) #check

["Éva Tardos (born 1 October 1957) is a Hungarian mathematician and the Jacob Gould Schurman Professor of Computer Science at Cornell University.\nTardos's research interest is algorithms. Her work focuses on the design and analysis of efficient methods for combinatorial optimization problems on graphs or networks. She has done some work on network flow algorithms like approximation algorithms for network flows, cut, and clustering problems. Her recent work focuses on algorithmic game theory and simple auctions.\n\n\n\nTardos received her Dipl. Math in 1981 and her Ph.D. 1984 from the Faculty of Sciences of the Eötvös Loránd University under her advisor András Frank. She was the Chair of the Department of Computer Science at Cornell from 2006-2010, and she is currently serving as the Associate Dean of the College of Computing and Information Science.\nShe was editor-in-Chief of SIAM Journal on Computing from 2004–2009, and is currently the Economics and Computation area editor of the J

In [ ]:
def check_entities_in_kg_graph(biography_text, kg_entities):
  #In this function, most of the steps will be commented for more clarity and better understanding

    spacy_doc = nlp(biography_text)
    spacy_entities = {(ent.start_char, ent.end_char): ent.label_ for ent in spacy_doc.ents}

    stanza_doc = stanza_nlp(biography_text)
    stanza_entities = {(ent.start_char, ent.end_char): ent.type for sent in stanza_doc.sentences for ent in sent.ents}


    matched_spacy_count = 0 # verify entity matches in KG graph with a counter
    matched_stanza_count = 0

#Use two loops like in the previous function:

    # First check for the Spacy entity matches in KG
    for span, entity_type in spacy_entities.items():
        entity_text = biography_text[span[0]:span[1]]  # Extract entity text using start-end positions
        pattern = re.compile(re.escape(entity_text), re.IGNORECASE)  # Create a regex pattern to match the entity text, case-insensitive
        for kg_entity in kg_entities:  # Iterate over KG entities
            if pattern.search(kg_entity):  # Check if the entity text matches any KG entity using regex search
                matched_spacy_count += 1  # Increment counter if a match is found
                break  # Exit the inner loop to avoid counting multiple matches for the same entity

    # Second, check for Stanza entity matches in KG
    for span, entity_type in stanza_entities.items():  # Iterate over Stanza entities with the exact same steps
        entity_text = biography_text[span[0]:span[1]]
        pattern = re.compile(re.escape(entity_text), re.IGNORECASE)
        for kg_entity in kg_entities:
            if pattern.search(kg_entity):
                matched_stanza_count += 1
                break

    # Calculate the ratio of predicted NEs found in the KG graph by dividing the count of matched entities by the total predicted entities to get the ratio.
    spacy_ratio = matched_spacy_count / len(spacy_entities) if spacy_entities else 0
    stanza_ratio = matched_stanza_count / len(stanza_entities) if stanza_entities else 0

    # Create a DataFrame
    df = pd.DataFrame([{
        'Matched in KG (SpaCy)': matched_spacy_count,
        'Matched in KG (Stanza)': matched_stanza_count,
        'Spacy KG Match Ratio': spacy_ratio,
        'Stanza KG Match Ratio': stanza_ratio
    }])

    return df


Unfortunately, the required KG-entities were not obtained, so the function could not be processed to see any output.